In [1]:
import pandas as pd

# importing the data 

In [ ]:
#a.1
dataframe = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
dataframe.head(10)

# understanding the dataset

In [ ]:
#a.2
dataframe.shape

In [ ]:
dataframe.columns

In [ ]:
#a.3
dataframe.describe()

# finding the missing value

In [ ]:
#a.4
dataframe.isnull().sum()

In [ ]:
#a.5
dataframe.duplicated().sum()

In [ ]:
dataframe.isna().sum()

In [ ]:
#dataframe.duplicated(keep=False) # show all duplicated row indexes

dataframe[dataframe.duplicated(keep=False)]

# Data cleaning:

In [10]:
# #b.1
# # handle the null values with the mean of the column
# # Exclude the column 'column_to_exclude'
# excluded_column = dataframe['Attrition']

# # Drop the column and fill NaN values with the mean for the remaining columns
# dataframe.drop(columns=['Attrition'], inplace=True)
# dataframe.fillna(dataframe.mean(), inplace=True)

# # Add the excluded column back to the dataframe
# dataframe['Attrition'] = excluded_column

In [ ]:
dataframe.isnull().sum()

In [ ]:
#b.2
dataframe.drop_duplicates(inplace=True)
dataframe

In [13]:
#b.3
dataframe.dropna(subset=['Attrition'], inplace=True)

# Creation of input and output features:

In [14]:
#c.1

Features = dataframe.drop('Attrition', axis=1)# 1 for column, 0 for row
Labels = dataframe['Attrition']

In [ ]:
Features

In [ ]:
Labels

# Conversion of features into numeric values:

In [ ]:
#d.1
from sklearn.preprocessing import LabelEncoder

distinct_values_count = dataframe['Attrition'].nunique()

if distinct_values_count == 2:
    encoder = LabelEncoder()
    Labels = encoder.fit_transform(Labels)
    Labels = pd.DataFrame(Labels, columns=['Attrition'])
else:
    Labels = pd.get_dummies(Features, columns=['Attrition'])


Labels#it is in array format
#better approach


In [ ]:
Features.dtypes

In [ ]:
#d.2
#catagorical_cols = [col for col in Features.columns if Features[col].dtype == 'object']
# Features= pd.get_dummies(Features, columns=catagorical_cols)

catagorical_cols = []
boolean_cols = []

for col in Features.columns:
    if Features[col].nunique()==2:
        boolean_cols.append(col)
        encoder = LabelEncoder()
        Features[col] = encoder.fit_transform(Features[col])# not sure about this line 
    else:
        if Features[col].dtype == 'object':
            # Features[col] = pd.get_dummies(Features[col], columns=[col])
            catagorical_cols.append(col)
            Features[col] = Features[col].astype('category')

print(catagorical_cols)
print(boolean_cols)

In [ ]:

Features

In [ ]:

#one hot encoding
Features = pd.get_dummies(Features, columns=catagorical_cols)
Features

# Scaling of the features

In [ ]:
#e
# there is a problem with the data info that while prediction it may give more 
# importance to the larger value of the target column
# we generally scale the features we dont scale the target column
#min-max scaling

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


scaling_cols = Features.columns.difference(catagorical_cols)
scaling_cols = scaling_cols.difference(boolean_cols)

def scale_min_max_features(Features):
    scaler = MinMaxScaler()
    for col in scaling_cols:
        Features[col] = scaler.fit_transform(Features[[col]])

def scale_standard_features(Features):
    scaler = StandardScaler()
    for col in scaling_cols:
        Features[col] = scaler.fit_transform(Features[[col]])

# i have to exclude one hot encoded columns from the scaling

scale_min_max_features(Features)
# scale_standard_features(Features)z

Features

# corelaiton scaling 

In [ ]:
#f.1
# correlation analysis of features with target
label_series = Labels['Attrition']
correlations = Features.corrwith(label_series).abs()
correlations


In [ ]:
Features_with_label = Features.copy()
Features_with_label['Attrition'] = label_series

correlation_matrix = Features_with_label.corr().abs()

correlation_matrix

In [ ]:
top_20_correlations = correlations.sort_values(ascending=False).head(20)

for col in top_20_correlations.index:
    print(f'{col} : {top_20_correlations[col]}')

In [ ]:
top_20_columns = top_20_correlations.index.tolist()

# Display the column names
top_20_columns

# 1D scatter plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 

# Separate the data based on numeric labels
class_0 = Features.loc[Labels["Attrition"] == 0]
class_1 = Features.loc[Labels["Attrition"] == 1]


def draw_plot(col_name, x_min=None, x_max=None):
    # Create a 1D scatter plot for sepal_width with numeric labels
    plt.plot(class_0[col_name], np.zeros_like(class_0[col_name]), 'o', label='Class No')
    plt.plot(class_1[col_name], np.zeros_like(class_1[col_name]), 'o', label='Class Yes')

    if x_min is not None and x_max is not None:
        plt.xlim(x_min, x_max)

    plt.legend()
    plt.xlabel(col_name)
    plt.title('1D Scatter Plot of '+col_name+' by Numeric Classes')
    plt.show()

for col in top_20_columns:  
    draw_plot(col,x_min=-1.5, x_max=1.5)


In [ ]:
# g.1
# create a simple classifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.1, random_state=42)

# Convert y_train and y_test to 1D arrays
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Create and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy